In [1]:
## information about how to use the script
from filmscope.recon_util import (add_sample_entry, get_all_sample_names,
                                  get_ss_volume_from_dataset)
from filmscope.util import play_video, load_dictionary
from filmscope.calibration import CalibrationInfoManager
from filmscope.datasets import FSDataset
import os
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
from filmscope.config import path_to_data

import torch

In [ ]:
# view already saved samples 
get_all_sample_names()

In [3]:
# optional: select GPU to be used
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [4]:
# necessary information about the dataset 
sample_name = "fluoro_bright_calib2" 
# this is the data's location within "path_to_data"
folder = "/" 
image_filename = folder +"/combo_fluoro_bright_dataset_1.nc"#'/finger_20240528.nc'

# frame number that will be viewed for videos
# set to -1 for images
frame_number = -1

calibration_filename = folder + '/calibration_information_fluor'
# blank images can be used to remove illumination artifacts
# set to None if this was not acquired
blank_filename = None # folder + "/blank_image.nc" 
bf = blank_filename 
if bf is not None:
    bf = path_to_data + '/' + bf

In [5]:
# some datasets were calibrated without saving the image shape used to calibrate
# this cell can be used to address that
manager = CalibrationInfoManager(path_to_data + '/' + calibration_filename)
if manager.image_shape is None:
    if manager.is_single_image:
        idx = manager.crop_indices[0]
        manager.image_shape = (idx[1] - idx[0], idx[3] - idx[2])
    else:
        print("adding image shape")
        dataset = xr.open_dataset(path_to_data + image_filename)
        downsample = int(dataset.y[1] - dataset.y[0])
        image_shape = (len(dataset.x) * downsample, len(dataset.y) * downsample)
        manager.image_shape = image_shape 
    manager.save_all_info()

In [ ]:
# set up a dataset with just the reference image 
ref_camera = CalibrationInfoManager(path_to_data + calibration_filename).reference_camera
downsample = 4
image_numbers = [ref_camera]
dataset = FSDataset(
    path_to_data + image_filename,
    path_to_data + calibration_filename,
    image_numbers,
    downsample,
    (0, 1, 0, 1),
    enforce_divisible=-1,
    frame_number=frame_number,
    blank_filename=bf
)

In [ ]:
# pick a crop level that shows the desired FOV
crop = (0, 1024, 0, 768) 
full_image = np.squeeze(dataset.reference_image)
crop_image = full_image[crop[0] : crop[1], crop[2] : crop[3]]

plt.imshow(crop_image)

plt.figure()
plt.imshow(full_image)
plt.colorbar()

In [ ]:
# determine crop values
shape0 = dataset.images.shape[2]
shape1 = dataset.images.shape[3]

crop_values = (
    crop[0] / shape0,
    crop[1] / shape0,
    crop[2] / shape1,
    crop[3] / shape1,
)
print(crop_values)

In [ ]:
dataset = None

# new dataset with the crops
downsample = 4
image_numbers = manager.image_numbers
dataset = FSDataset(
    path_to_data + image_filename,
    path_to_data + calibration_filename,
    image_numbers,
    downsample,
    crop_values,
    enforce_divisible=-1,
    frame_number=frame_number,
    blank_filename=bf
)

In [ ]:
plt.imshow(dataset.images[0].squeeze())


In [23]:
# find depth extents
low_plane = -8
high_plane = 10

num = 30
depths = np.linspace(low_plane, high_plane, num, endpoint=True)
depth_values = torch.from_numpy(depths).to(torch.float32).cuda()

batch_size = 10
volume = get_ss_volume_from_dataset(dataset, batch_size, depth_values, get_squared=False)
volume = volume.squeeze().cpu().numpy()

In [ ]:
play_video(volume)

In [ ]:
idx = 0
plt.imshow(volume[idx], cmap='gray')
plt.title("{:.2f} mm".format(depths[idx]))

In [26]:
# make the full sample entry
info = {
    "folder": folder,
    "sample_name": sample_name,
    "calibration_filename": calibration_filename,
    "image_filename": image_filename,
    "crop_values": crop_values,
    "depth_range": (low_plane, high_plane),
    "blank_filename": blank_filename,
}

In [ ]:
# and save it!
# this cell is commented out for the demo

add_sample_entry(sample_name, info, overwrite=False)